In [1]:
import pickle
import numpy as np


def unpickle(fileName):
    '''
    Description: retrieve data from CIFAR-10 Pickles
    Params: fileName = filename to unpickle
    Outputs: Unpickled Dict
    '''
    with open(fileName, 'rb') as f:
        dict = pickle.load(f, encoding='bytes')
    return dict


def merge_batches(num_to_load=1):
        fileName = "cifar-10-batches-py/data_batch_1"
        data = unpickle(fileName)
        features = data[b"data"]
        labels = np.array(data[b"labels"])
        
        fileName2 = "cifar-10-batches-py/data_batch_2"
        data2 = unpickle(fileName2)
        features = np.append(features, data2[b"data"], axis=0)
        labels = np.append(labels, data2[b"labels"], axis=0)
        
        fileName3 = "cifar-10-batches-py/data_batch_3"
        data3 = unpickle(fileName3)
        features = np.append(features, data3[b"data"], axis=0)
        labels = np.append(labels, data3[b"labels"], axis=0)
        
        fileName4 = "cifar-10-batches-py/data_batch_4"
        data4 = unpickle(fileName4)
        features = np.append(features, data4[b"data"], axis=0)
        labels = np.append(labels, data4[b"labels"], axis=0)
        
        fileName5 = "cifar-10-batches-py/data_batch_5"
        data5 = unpickle(fileName5)
        features = np.append(features, data5[b"data"], axis=0)
        labels = np.append(labels, data5[b"labels"], axis=0)
        return features, labels


def one_hot_encode(data):
    '''
    Description: Encode Target Label IDs to one hot vector of size L where L is the
    number of unique labels
    Params: data = list of label IDs
    Outputs: List of One Hot Vectors
    '''
    one_hot = np.zeros((data.shape[0], 10))
    one_hot[np.arange(data.shape[0]), data] = 1
    return one_hot


def normalize(data):
    '''
    Description: Normalize Pixel values
    Params: list of Image Pixel Features
    Outputs: Normalized Image Pixel Features
    '''
    return data / 255.0


def preprocess(num_to_load=1):
    '''
    Description: helper function to load and preprocess CIFAR-10 training data batches
    Params: num_to_load = number of batches of CIFAR-10 to load and merge
    Outputs: Pre-processed CIFAR-10 image features and labels
    '''
    X, y = merge_batches(num_to_load=1)
    X = normalize(X)
    X = X.reshape(-1, 3072)
    y = one_hot_encode(y)
    y = y.reshape(-1, 10)
    return X, y


In [2]:
import numpy as np


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T

    x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x))


def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    # only multiply with the index of t (target) with value '1'
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size


def softmax_loss(X, t):
    y = softmax(X)
    return cross_entropy_error(y, t)

class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx


class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b

        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        dx = dx.reshape(*self.original_x_shape)
        return dx


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size:
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size

        return dx

In [3]:
from collections import OrderedDict

class Mlp:
    def __init__(self, input_size, hidden_size, hidden_size1, output_size, weight_init_std=0.01):
        self.params = {}
        
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, hidden_size1)
        self.params['b2'] = np.zeros(hidden_size1)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size1, output_size)
        self.params['b3'] = np.zeros(output_size)

        self.layers = OrderedDict()
        # define affine layer
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Sigmoid1'] = Sigmoid()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Sigmoid2'] = Sigmoid()
        self.layers['Affine3'] = Affine(self.params['W3'], self.params['b3'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        grads['W3'], grads['b3'] = self.layers['Affine3'].dW, self.layers['Affine3'].db
        return grads

In [4]:
def train(model, x_train, y_train, x_test, y_test, learning_rate=0.1, epochs=30, batch_size=32):
    train_size = x_train.shape[0]
    print("Training starts")
    for i in range(epochs):
        x_batches = [
            x_train[k:k + batch_size]
            for k in range(0, train_size, batch_size)]
        y_batches = [
            y_train[k:k + batch_size]
            for k in range(0, train_size, batch_size)]

        for x_batch, y_batch in zip(x_batches, y_batches):
            grad = model.gradient(x_batch, y_batch)
            # update weights and biases
            for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3'):
                model.params[key] -= learning_rate * grad[key]

        loss = model.loss(x_train, y_train)
        train_loss_list.append(loss)

        train_acc = model.accuracy(x_train, y_train)
        test_acc = model.accuracy(x_test, y_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc:", train_acc*100, "   test acc:", test_acc*100)

In [5]:
X_train, y_train = preprocess(num_to_load=1)

In [6]:
X_test = unpickle("cifar-10-batches-py/test_batch")[b"data"] / 255.0
y_test = unpickle("cifar-10-batches-py/test_batch")[b"labels"]
X_test = X_test.reshape(-1, 3072)
y_test = np.array(y_test)
y_test = one_hot_encode(y_test)
y_test = y_test.reshape(-1, 10)
print(X_test.shape)
print(y_test.shape)

(10000, 3072)
(10000, 10)


In [7]:
    train_loss_list = []
    train_acc_list = []
    test_acc_list = []
    train_acc_avg = []
    test_acc_avg = []
model = Mlp(3072, 256, 256, 10)
train(model, X_train, y_train, X_test, y_test, learning_rate=0.1, epochs=30, batch_size=32)

Training starts
train acc: 16.298000000000002    test acc: 16.56
train acc: 19.8    test acc: 19.919999999999998
train acc: 25.384    test acc: 25.230000000000004
train acc: 26.642    test acc: 26.779999999999998
train acc: 30.988    test acc: 31.080000000000002
train acc: 34.066    test acc: 33.93
train acc: 36.86    test acc: 36.84
train acc: 37.85    test acc: 37.76
train acc: 38.744    test acc: 39.09
train acc: 39.796    test acc: 39.86
train acc: 40.682    test acc: 40.71
train acc: 40.952    test acc: 40.69
train acc: 40.239999999999995    test acc: 39.95
train acc: 39.668    test acc: 39.300000000000004
train acc: 39.562000000000005    test acc: 38.91
train acc: 39.858    test acc: 39.04
train acc: 39.95    test acc: 38.96
train acc: 39.852    test acc: 38.74
train acc: 39.946    test acc: 38.800000000000004
train acc: 40.254    test acc: 39.14
train acc: 40.71    test acc: 39.489999999999995
train acc: 41.448    test acc: 39.77
train acc: 42.370000000000005    test acc: 40.510